<a href="https://colab.research.google.com/github/DionKimmm/2019SummerML/blob/master/Day23_03_RNN_BasicLSTMCell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![대체 텍스트](http://drive.google.com/uc?export=view&id=1-yNJvsVyZCFvOONiyv0Ra0GKqzHs-PwS)

In [0]:
import warnings
warnings.filterwarnings(action='ignore') 

In [0]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

In [3]:
# sample = "The grass is always greener on the other side of the fence."
sample = "낮 말은 새가 듣고 밤 말은 쥐가 듣는다"
# sample = "남의 떡이 더 커 보이고 남이 잡은 일감이 더 헐어 보인다."
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex
print(idx2char)
print(char2idx)

[' ', '밤', '낮', '다', '새', '고', '쥐', '는', '가', '은', '말', '듣']
{' ': 0, '밤': 1, '낮': 2, '다': 3, '새': 4, '고': 5, '쥐': 6, '는': 7, '가': 8, '은': 9, '말': 10, '듣': 11}


In [0]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

In [5]:
sample_idx = [char2idx[c] for c in sample]  # char to index
print(sample_idx)
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
print(x_data)
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello
print(y_data)

[2, 0, 10, 9, 0, 4, 8, 0, 11, 5, 0, 1, 0, 10, 9, 0, 6, 8, 0, 11, 7, 3]
[[2, 0, 10, 9, 0, 4, 8, 0, 11, 5, 0, 1, 0, 10, 9, 0, 6, 8, 0, 11, 7]]
[[0, 10, 9, 0, 4, 8, 0, 11, 5, 0, 1, 0, 10, 9, 0, 6, 8, 0, 11, 7, 3]]


In [0]:
tf.reset_default_graph()

In [0]:
X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [8]:
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

W0726 11:05:35.559121 140290909833088 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0726 11:05:35.561034 140290909833088 deprecation.py:323] From <ipython-input-8-552346adc015>:3: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0726 11:05:35.580603 140290909833088 deprecation.py:323] From <ipython-input-8-552346adc015>:6: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please 

In [0]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected\
        (X_for_fc, num_classes, activation_fn=None)

In [0]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [0]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [0]:
prediction = tf.argmax(outputs, axis=2)

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.5095203 Prediction:                      
1 loss: 2.3214223 Prediction:                      
2 loss: 2.1264899 Prediction:                      
3 loss: 2.0504367 Prediction:                      
4 loss: 1.8998662 Prediction:                      
5 loss: 1.7392294 Prediction:        듣고   말     듣고다
6 loss: 1.5304724 Prediction:        듣고 말 말     듣고다
7 loss: 1.2960448 Prediction:  말은    듣고 말 말은    듣고다
8 loss: 1.064335 Prediction:  말은  가 듣고 말 말은 은가 듣는다
9 loss: 0.8342992 Prediction:  말은 새가 듣는 밤 말은 은가 듣는다
10 loss: 0.6083646 Prediction:  말은 새가 듣는 밤 말은 쥐가 듣는다
11 loss: 0.44654456 Prediction:  말은 새가 듣는 밤 말은 쥐가 듣는다
12 loss: 0.33415475 Prediction:  말은 새가 듣는 밤 말은 쥐가 듣는다
13 loss: 0.2552515 Prediction:  말은 새가 듣고 밤 말은 쥐가 듣고다
14 loss: 0.20063464 Prediction:  말은 새가 듣고 밤 말은 쥐가 듣고다
15 loss: 0.16205163 Prediction:  말은 새가 듣고 밤 말은 쥐가 듣고다
16 loss: 0.13069604 Prediction:  말은 새가 듣고 밤 말은 쥐가 듣는다
17 loss: 0.11061658 Prediction:  말은 새가 듣고 밤 말은 새가 듣는다
18 loss: 0.110831305 Prediction:  말은 새가 듣고 밤 말은 쥐가 